## Submit MRI Import

This notebook submits MRI Import jobs to Moody's Risk Modeler

### Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.batch import submit_batch, get_batch_jobs, read_batch
from helpers.database import execute_query
from helpers.step import get_last_step_run
from helpers.irp_integration import IRPClient
from helpers.constants import BatchType

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_03_Submit_MRI_Import_Batch.ipynb')

# Display context
ux.header("Submit MRI Import Batch")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"✓ Step tracking initialized for '{context.step_name}'")

### Retrieve MRI Import Batch

In [ ]:
# Retrieve MRI Import Creation batch from Stage_01/Step_03
ux.subheader("Retrieve MRI Import Batch")

# Query for Stage_01/Step_03 step run
query = """
    SELECT sr.id, sr.step_id, sr.run_num, sr.output_data, sr.completed_ts
    FROM irp_step_run sr
    INNER JOIN irp_step s ON sr.step_id = s.id
    INNER JOIN irp_stage sg ON s.stage_id = sg.id
    INNER JOIN irp_cycle c ON sg.cycle_id = c.id
    WHERE c.cycle_name = %s
      AND sg.stage_num = 1
      AND s.step_num = 3
      AND sr.status = 'COMPLETED'
    ORDER BY sr.completed_ts DESC
    LIMIT 1
"""

result = execute_query(query, (context.cycle_name,))

if result.empty:
    raise ValueError("Batch creation step not found - please complete Stage_01/Step_03 first")

output_data = result.iloc[0]['output_data']
batches = output_data.get('batches', {})

if BatchType.MRI_IMPORT not in batches:
    raise ValueError(f"MRI Import batch not found. Available: {list(batches.keys())}")

mri_batch_id = int(batches[BatchType.MRI_IMPORT])

ux.success(f"✓ Retrieved MRI Import batch: ID={mri_batch_id}")
step.log(f"Retrieved MRI Import batch: ID={mri_batch_id}")

### Review MRI Import Batch Configuration

In [ ]:
# Verify batch status and display job information
ux.subheader("Verify Batch Status")

# Read batch details
batch = read_batch(mri_batch_id)

batch_info = [
    ["Batch ID", batch['id']],
    ["Batch Type", batch['batch_type']],
    ["Status", batch['status']],
    ["Created", batch['created_ts'].strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(batch_info, headers=["Property", "Value"])

# Get jobs in batch
jobs = get_batch_jobs(mri_batch_id)
job_count = len(jobs)

ux.info(f"\nTotal jobs: {job_count}")

# Show sample configurations
if job_count > 0:
    ux.info("\nSample MRI Import Configurations (first 5):")
    rows = []
    for i, job in enumerate(jobs[:5]):
        config_query = "SELECT job_configuration_data FROM irp_job_configuration WHERE id = %s"
        config_result = execute_query(config_query, (job['job_configuration_id'],))
        if not config_result.empty:
            config = config_result.iloc[0]['job_configuration_data']
            rows.append([
                config.get('Portfolio', 'N/A'),
                config.get('Database', 'N/A'),
                config.get('accounts_import_file', 'N/A'),
                config.get('locations_import_file', 'N/A'),
            ])
    ux.table(rows, headers=["Portfolio", "EDM", "Accounts File", "Locations File"])

step.log(f"Verified batch: {job_count} jobs ready for submission")

### Submit MRI Import Batch to Moody's

In [ ]:
# Submit batch to Moody's API
ux.subheader("Submit Batch to Moody's")

ux.info("")
ux.info("Submission Process:")
# ux.info("  • Each job will lookup/create an exposure set in Moody's")
# ux.info("  • Database server 'databridge-1' will be used for all EDMs")
# ux.info("  • Jobs will transition to SUBMITTED status")
# ux.info("  • Batch will transition to ACTIVE status")
ux.info("")

# Submit
ux.info("\nSubmitting batch...")

# Pass step.step_id to associate batch with this step (not the creation step)
result = submit_batch(mri_batch_id, IRPClient(), step_id=step.step_id)

# Display results
ux.success(f"\n✓ Batch submission completed")
ux.info(f"  Submitted: {result['submitted_jobs']} jobs")
ux.info(f"  Status: {result['batch_status']}")

# Check for errors
failed_count = len([j for j in result['jobs'] if 'error' in j])
if failed_count > 0:
    ux.warning(f"\n⚠ {failed_count} job(s) failed to submit")
    for job_result in result['jobs']:
        if 'error' in job_result:
            ux.error(f"  Job {job_result['job_id']}: {job_result['error']}")

step.log(f"Batch submitted: {result['submitted_jobs']} jobs, {failed_count} failed")

### Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

try:
    # Prepare output data
    output_data = {
        'batch_id': mri_batch_id,
        'batch_type': batch['batch_type'],
        'batch_status': result['batch_status'],
        'submitted_jobs': result['submitted_jobs'],
        'failed_jobs': failed_count
    }
    
    # Complete the step
    step.complete(output_data)

    ux.success("\n" + "="*60)
    ux.success("✓ MRI IMPORT BATCH SUBMITTED SUCCESSFULLY")
    ux.success("="*60)
    ux.info(f"\nSubmitted {result['submitted_jobs']} job(s) to Moody's API")
    if failed_count > 0:
        ux.warning(f"⚠ {failed_count} job(s) failed to submit")
    ux.info(f"Batch status: {result['batch_status']}")
    ux.info("Next: Proceed to Reinsurance Treaty Creation or other data loading steps")

except Exception as e:
    ux.error(f"✗ Step completion failed: {str(e)}")
    step.fail(str(e))
    raise